In [2]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [7]:

from google import genai
from google.genai import types

# Directories
image_dir = "/content/Ingredients_Images/"  # your images
desc_dir = "/content/Ingredients/"
recipe_dir = "/content/Recipes/"
os.makedirs(desc_dir, exist_ok=True)
os.makedirs(recipe_dir, exist_ok=True)

# Initialize Gemma client
client = genai.Client()
image_model = "gemma-3-27b-it"

# Loop over images
for filename in os.listdir(image_dir):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(image_dir, filename)

        # Step 1: Extract ingredients
        with open(image_path, "rb") as f:
            image_bytes = f.read()

        contents = [
            types.Content(
                role="user",
                parts=[
                    types.Part.from_text(
                        text="Please analyze this image and list all ingredients with approximate quantities. Example format: '1 onion, 2 eggs, 500 grams flour'."
                    ),
                    types.Part.from_bytes(data=image_bytes, mime_type="image/jpeg"),
                ],
            )
        ]

        generate_content_config = types.GenerateContentConfig()
        ingredients_text = ""
        try:
            for chunk in client.models.generate_content_stream(
                model=image_model,
                contents=contents,
                config=generate_content_config,
            ):
                if chunk.text:
                    ingredients_text += chunk.text
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

        # Save ingredient description
        desc_path = os.path.join(desc_dir, os.path.splitext(filename)[0] + "_ingredients.txt")
        with open(desc_path, "w", encoding="utf-8") as f:
            f.write(ingredients_text)

        print(f"✅ Ingredients for {filename}: {ingredients_text}")

        # Step 2: Generate recipes for this image
        recipe_prompt = f"""
        I have the following ingredients: {ingredients_text}.
        Please provide 2-3 different recipes using these ingredients. Include:
        - Recipe name
        - Ingredients list
        - Step-by-step instructions
        - Cooking time and servings if possible
        """

        contents = [
            types.Content(
                role="user",
                parts=[types.Part.from_text(text=recipe_prompt)]
            )
        ]

        recipes_text = ""
        try:
            for chunk in client.models.generate_content_stream(
                model=image_model,
                contents=contents,
                config=types.GenerateContentConfig(),
            ):
                if chunk.text:
                    recipes_text += chunk.text
        except Exception as e:
            print(f"Error generating recipes for {filename}: {e}")
            continue

        # Save recipes
        recipe_path = os.path.join(recipe_dir, os.path.splitext(filename)[0] + "_recipes.txt")
        with open(recipe_path, "w", encoding="utf-8") as f:
            f.write(recipes_text)

        print(f"✅ Recipes for {filename} saved to {recipe_path}")

✅ Ingredients for 4.png: Here's an analysis of the ingredients in the image, with approximate quantities:

*   1 red onion
*   1 head of broccoli
*   2-3 carrots
*   1 leek
*   250 grams cherry tomatoes
*   250 grams pasta (likely spaghetti)
*   1 can (approx. 400g) chickpeas
*   2 slices of bread
*   2 apples
*   250 ml olive oil
*   3-4 potatoes
*   6 eggs
*   1 avocado
*   2 chicken breasts (approx. 300-400g total)
*   1 head of garlic
*   1 bunch of parsley
*   100g spinach
*   100g mushrooms
*   100g bacon
*   100g couscous
*   100g bulgur
*   1 jar (approx. 200-300g) tomato paste/sauce
*   1 block of tofu (approx. 200-300g)
*   100g butter
*   100g cheddar cheese
*   1 green bell pepper
*   Salt and pepper (to taste, visible as shakers)

**Please note:** These are estimates based on visual assessment. Actual quantities may vary.
✅ Recipes for 4.png saved to /content/Recipes/4_recipes.txt
✅ Ingredients for 2.png: Here's an analysis of the ingredients in the image with approximate 